<a href="https://colab.research.google.com/github/jpaulobf/XPE-CRF2/blob/main/An%C3%A1lise_atas_Copom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
pd.set_option("display.max_colwidth", 150)
pd.set_option("display.min_rows", 20)

import urllib.request, json

1. Buscando dados

In [ ]:
import json

In [ ]:
url = 'https://www.bcb.gov.br/api/servico/sitebcb/atascopom/ultimas?quantidade=1000&filtro='

In [ ]:
import urllib.request, json
with urllib.request.urlopen(url) as url:
    data = json.load(url)
    print(data)

In [ ]:
data['conteudo'][0:3]

In [ ]:
df = pd.DataFrame(data['conteudo'])

In [ ]:
df.head()

In [ ]:
df.iloc[0]['LinkPagina'][-4:]

'2023'

In [ ]:
df[df.LinkPagina.str.endswith('2021')]

In [ ]:
df[(df.LinkPagina.str.endswith('2021')) | (df.LinkPagina.str.endswith('2020')) | (df.LinkPagina.str.endswith('2019'))]

In [ ]:
lista_urls = df[(df.LinkPagina.str.endswith('2021')) | (df.LinkPagina.str.endswith('2020')) | (df.LinkPagina.str.endswith('2019'))]['Url'].to_list()

In [ ]:
lista_urls

In [ ]:
def download_pdfs(lista_urls):

    lista_arquivos = []

    for url in lista_urls:
        nome_arquivo = url[url.rfind("/") + 1:]
        urllib.request.urlretrieve("https://www.bcb.gov.br" + url, nome_arquivo)
        lista_arquivos.append(nome_arquivo)

    return lista_arquivos

In [ ]:
lista_arquivos = download_pdfs(lista_urls)

In [ ]:
lista_arquivos

In [ ]:
!pip install pdfplumber

In [ ]:
import pdfplumber

In [ ]:
lista_textos = []
lista_nomes = []

In [ ]:
def pdf_to_txt(lista_arquivos):

    for arquivo in lista_arquivos:

        with pdfplumber.open("./" + arquivo) as pdf:
            paginas = pdf.pages[2:]
            texto = ""

            for pagina in paginas:
                texto += pagina.extract_text()
                texto.encode('utf-8', 'ignore').decode('utf-8')

            lista_textos.append(texto)
            lista_nomes.append(arquivo)

            #with open(arquivo.replace('.pdf', '.txt'), 'w') as f:
            #    f.write(texto)

In [ ]:
pdf_to_txt(lista_arquivos[:-1])

### Análise

In [ ]:
from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize
import nltk

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
arquivo = "Copom243-not20211208243.txt"

In [44]:
texto = ""
#with open(arquivo, 'r') as f:
#    texto = f.read()
#texto

In [45]:
#texto = ""
#with open(arquivo, 'r') as f:
#    texto = f.read().replace('\n', ' ').lower()

In [ ]:
texto

In [ ]:
frequency_distribution = FreqDist(word.lower() for word in word_tokenize(texto))

In [ ]:
frequency_distribution.most_common(20)

In [ ]:
nltk.download('stopwords')

In [ ]:
stopwords = nltk.corpus.stopwords.words('portuguese')

In [ ]:
frequency_distribution = FreqDist(word.lower() for word in word_tokenize(texto) if word not in stopwords)

In [ ]:
frequency_distribution.most_common(20)

[]

In [ ]:
from string import punctuation
lista_adicional = ['2019', '2020', '2021', '2022', '2023', 'copom', 'comitê']
stopwords = set(list(stopwords) + list(punctuation) + lista_adicional)

In [ ]:
frequency_distribution = FreqDist(word.lower() for word in word_tokenize(texto) if word.lower() not in stopwords)
frequency_distribution.most_common(20)

In [ ]:
dict_top = {}
counter = 0

#for arquivo in lista_arquivos:
#    with open(arquivo, 'r') as f:
#        texto = f.read().replace('\n', ' ').lower()
#        frequency_distribution = FreqDist(word for word in word_tokenize(texto) if word not in stopwords)
#        dict_top[arquivo.replace('.txt', '')] = frequency_distribution.most_common(10)


for arquivo in lista_textos:
    texto = arquivo.replace('\n', ' ').lower()
    frequency_distribution = FreqDist(word for word in word_tokenize(texto) if word not in stopwords)
    dict_top[lista_nomes[counter]] = frequency_distribution.most_common(10)
    counter = counter + 1

In [ ]:
df = pd.DataFrame(dict_top)

In [ ]:
df.T.sort_index().head(25)

Bigramas

In [ ]:
dict_top_bigram = {}
counter = 0

#for arquivo in lista_arquivos:

    #with open(arquivo, 'r') as f:
    #    texto = f.read().replace('\n', ' ').lower()
    #    frequency_distribution = FreqDist(word for word in nltk.bigrams([word for word in word_tokenize(texto) if word not in stopwords]))
    #    dict_top_bigram[arquivo.replace('.txt', '')] = frequency_distribution.most_common(10)

for arquivo in lista_textos:
    texto = arquivo.replace('\n', ' ').lower()
    frequency_distribution = FreqDist(word for word in nltk.bigrams([word for word in word_tokenize(texto) if word not in stopwords]))
    dict_top_bigram[lista_nomes[counter]] = frequency_distribution.most_common(10)
    counter = counter + 1

In [ ]:
df = pd.DataFrame(dict_top_bigram)
df.T.sort_index().head(25)

Trigramas

In [ ]:
dict_top_trigram = {}
counter = 0

#for arquivo in lista_arquivos:

    #with open(arquivo, 'r') as f:
    #    texto = f.read().replace('\n', ' ').lower()
    #    frequency_distribution = FreqDist(word for word in nltk.trigrams([word for word in word_tokenize(texto) if word not in stopwords]))
    #    dict_top_trigram[arquivo.replace('.txt', '')] = frequency_distribution.most_common(10)

for arquivo in lista_textos:
    texto = arquivo.replace('\n', ' ').lower()
    frequency_distribution = FreqDist(word for word in nltk.trigrams([word for word in word_tokenize(texto) if word not in stopwords]))
    dict_top_trigram[lista_nomes[counter]] = frequency_distribution.most_common(10)
    counter = counter + 1

In [ ]:
df = pd.DataFrame(dict_top_trigram)
df.T.sort_index().head(25)